In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import time
from itertools import islice

In [13]:
# Create DataFrame from CSV
t0 = time.time()
filtered_cooccurring_df = pd.read_csv('../../Data/filtered_cooccurring_TF.csv')
filtered_cooccurring_df.index = filtered_cooccurring_df.index - 1
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
filtered_cooccurring_df.head(3)

Run time: 45.562724113464355 seconds


,song_name,artist_name,category_name,category_id,genre_list,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_mode,audio_ft_speechiness,...,good time,dont mind,last night,cause youre,dont worry,brand new,like dont,know like,im feeling,feel good
-1,NaN,NaN,NaN,53468.0,NaN,4528.6433,5362.4272,42651.0,5517.0,665.7547,...,259.0,252.0,257.0,251.0,256.0,255.0,254.0,253.0,239.0,240.0
0,willow,Taylor Swift,pop,8.0,"['dance', 'pop']",0.3920,0.5740,7.0,1.0,0.1700,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Stay Next To Me (with Chelsea Cutler),Quinn XCII,pop,8.0,"['indie', 'pop', 'electropop']",0.5810,0.5840,2.0,1.0,0.2840,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# Create a dictionary of category names and ids
cat_name_id = {}
category_list = ['blues', 'classical', 'country', 'funk', 'hiphop', 'indie_alt', 'jazz', 
                 'metal', 'pop', 'punk', 'rnb', 'rock', 'romance', 'soul']
for cat in category_list:
    cat_id = list(filtered_cooccurring_df[filtered_cooccurring_df['category_name']==cat]['category_id'])[0]
    cat_name_id[cat] = cat_id
    print(cat, cat_id)

blues 0.0
classical 1.0
country 2.0
funk 3.0
hiphop 4.0
indie_alt 5.0
jazz 6.0
metal 7.0
pop 8.0
punk 9.0
rnb 10.0
rock 11.0
romance 12.0
soul 13.0


In [32]:
# Create a new DataFrame with co-occurring word columns
new_lyric_TF_df = filtered_cooccurring_df.copy()
new_lyric_TF_df = new_lyric_TF_df.drop(columns=new_lyric_TF_df.columns[17:14144])
new_lyric_TF_df.head(3)

,song_name,artist_name,category_name,category_id,genre_list,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_mode,audio_ft_speechiness,...,good time,dont mind,last night,cause youre,dont worry,brand new,like dont,know like,im feeling,feel good
-1,NaN,NaN,NaN,53468.0,NaN,4528.6433,5362.4272,42651.0,5517.0,665.7547,...,259.0,252.0,257.0,251.0,256.0,255.0,254.0,253.0,239.0,240.0
0,willow,Taylor Swift,pop,8.0,"['dance', 'pop']",0.3920,0.5740,7.0,1.0,0.1700,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Stay Next To Me (with Chelsea Cutler),Quinn XCII,pop,8.0,"['indie', 'pop', 'electropop']",0.5810,0.5840,2.0,1.0,0.2840,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
# Create a list of the overall top ten words 
word_count_df = new_lyric_TF_df[0:1][new_lyric_TF_df.columns[17:]]
word_count_df = word_count_df.sort_values(by=[-1], axis=1, ascending=False)
top_word_counts = list(word_count_df.loc[-1])[0:10]
top_ten = dict(zip(list(word_count_df)[0:10], top_word_counts))
top_ten

{'oh oh': 4588.0,
 'yeah yeah': 3169.0,
 'ooh ooh': 2568.0,
 'na na': 2425.0,
 'dont know': 2363.0,
 'la la': 2323.0,
 'dont wanna': 1487.0,
 'let go': 1215.0,
 'im gonna': 1200.0,
 'oh yeah': 1018.0}

In [34]:
# Create a dictionary of the top ten co-occurring words for each category
t0 = time.time()
top_words = {}
df_rows = []
blues_df = new_lyric_TF_df[new_lyric_TF_df['category_name']=='blues'].copy()
blues_df.loc[0] = blues_df.sum(numeric_only=True)
blues_words = blues_df.sort_index()[blues_df.columns[17:]][0:1]
blues_words = blues_words.sort_values(by=[0], axis=1, ascending=False)
top_word_counts = list(blues_words.loc[0])[0:10]
top_ten = dict(zip(list(blues_words)[0:10], top_word_counts))
top_words['blues'] = top_ten
df_rows.append(list(top_ten.keys()))
df_rows.append(list(top_ten.values()))

classical_df = new_lyric_TF_df[new_lyric_TF_df['category_name']=='classical'].copy()
classical_df.loc[0] = classical_df.sum(numeric_only=True)
classical_words = classical_df.sort_index()[classical_df.columns[17:]][0:1]
classical_words = classical_words.sort_values(by=[0], axis=1, ascending=False)
top_word_counts = list(classical_words.loc[0])[0:10]
top_ten = dict(zip(list(classical_words), top_word_counts))
top_words['classical'] = top_ten
df_rows.append(list(top_ten.keys()))
df_rows.append(list(top_ten.values()))

country_df = new_lyric_TF_df[new_lyric_TF_df['category_name']=='country'].copy()
country_df.loc[0] = country_df.sum(numeric_only=True)
country_words = country_df.sort_index()[country_df.columns[17:]][0:1]
country_words = country_words.sort_values(by=[0], axis=1, ascending=False)
top_word_counts = list(country_words.loc[0])[0:10]
top_ten = dict(zip(list(country_words)[0:10], top_word_counts))
top_words['country'] = top_ten
df_rows.append(list(top_ten.keys()))
df_rows.append(list(top_ten.values()))

funk_df = new_lyric_TF_df[new_lyric_TF_df['category_name']=='funk'].copy()
funk_df.loc[0] = funk_df.sum(numeric_only=True)
funk_words = funk_df.sort_index()[funk_df.columns[17:]][0:1]
funk_words = funk_words.sort_values(by=[0], axis=1, ascending=False)
top_word_counts = list(funk_words.loc[0])[0:10]
top_ten = dict(zip(list(funk_words)[0:10], top_word_counts))
top_words['funk'] = top_ten
df_rows.append(list(top_ten.keys()))
df_rows.append(list(top_ten.values()))

hiphop_df = new_lyric_TF_df[new_lyric_TF_df['category_name']=='hiphop'].copy()
hiphop_df.loc[0] = hiphop_df.sum(numeric_only=True)
hiphop_words = hiphop_df.sort_index()[hiphop_df.columns[17:]][0:1]
hiphop_words = hiphop_words.sort_values(by=[0], axis=1, ascending=False)
top_word_counts = list(hiphop_words.loc[0])[0:10]
top_ten = dict(zip(list(hiphop_words)[0:10], top_word_counts))
top_words['hiphop'] = top_ten
df_rows.append(list(top_ten.keys()))
df_rows.append(list(top_ten.values()))

indie_alt_df = new_lyric_TF_df[new_lyric_TF_df['category_name']=='indie_alt'].copy()
indie_alt_df.loc[0] = indie_alt_df.sum(numeric_only=True)
indie_alt_words = indie_alt_df.sort_index()[indie_alt_df.columns[17:]][0:1]
indie_alt_words = indie_alt_words.sort_values(by=[0], axis=1, ascending=False)
top_word_counts = list(indie_alt_words.loc[0])[0:10]
top_ten = dict(zip(list(indie_alt_words)[0:10], top_word_counts))
top_words['indie_alt'] = top_ten
df_rows.append(list(top_ten.keys()))
df_rows.append(list(top_ten.values()))

jazz_df = new_lyric_TF_df[new_lyric_TF_df['category_name']=='jazz'].copy()
jazz_df.loc[0] = jazz_df.sum(numeric_only=True)
jazz_words = jazz_df.sort_index()[jazz_df.columns[17:]][0:1]
jazz_words = jazz_words.sort_values(by=[0], axis=1, ascending=False)
top_word_counts = list(jazz_words.loc[0])[0:10]
top_ten = dict(zip(list(jazz_words)[0:10], top_word_counts))
top_words['jazz'] = top_ten
df_rows.append(list(top_ten.keys()))
df_rows.append(list(top_ten.values()))

metal_df = new_lyric_TF_df[new_lyric_TF_df['category_name']=='metal'].copy()
metal_df.loc[0] = metal_df.sum(numeric_only=True)
metal_words = metal_df.sort_index()[metal_df.columns[17:]][0:1]
metal_words = metal_words.sort_values(by=[0], axis=1, ascending=False)
top_word_counts = list(metal_words.loc[0])[0:10]
top_ten = dict(zip(list(metal_words)[0:10], top_word_counts))
top_words['metal'] = top_ten
df_rows.append(list(top_ten.keys()))
df_rows.append(list(top_ten.values()))

pop_df = new_lyric_TF_df[new_lyric_TF_df['category_name']=='pop'].copy()
pop_df.loc[-1] = pop_df.sum(numeric_only=True)
pop_words = pop_df.sort_index()[pop_df.columns[17:]]
pop_words = pop_words.sort_values(by=[0], axis=1, ascending=False)
top_word_counts = list(pop_words.loc[0])[0:10]
top_ten = dict(zip(list(pop_words)[0:10], top_word_counts))
top_words['pop'] = top_ten
df_rows.append(list(top_ten.keys()))
df_rows.append(list(top_ten.values()))

punk_df = new_lyric_TF_df[new_lyric_TF_df['category_name']=='punk'].copy()
punk_df.loc[0] = punk_df.sum(numeric_only=True)
punk_words = punk_df.sort_index()[punk_df.columns[17:]][0:1]
punk_words = punk_words.sort_values(by=[0], axis=1, ascending=False)
top_word_counts = list(punk_words.loc[0])[0:10]
top_ten = dict(zip(list(punk_words)[0:10], top_word_counts))
top_words['punk'] = top_ten
df_rows.append(list(top_ten.keys()))
df_rows.append(list(top_ten.values()))

rnb_df = new_lyric_TF_df[new_lyric_TF_df['category_name']=='rnb'].copy()
rnb_df.loc[0] = rnb_df.sum(numeric_only=True)
rnb_words = rnb_df.sort_index()[rnb_df.columns[17:]][0:1]
rnb_words = rnb_words.sort_values(by=[0], axis=1, ascending=False)
top_word_counts = list(rnb_words.loc[0])[0:10]
top_ten = dict(zip(list(rnb_words)[0:10], top_word_counts))
top_words['rnb'] = top_ten
df_rows.append(list(top_ten.keys()))
df_rows.append(list(top_ten.values()))

rock_df = new_lyric_TF_df[new_lyric_TF_df['category_name']=='rock'].copy()
rock_df.loc[0] = rock_df.sum(numeric_only=True)
rock_words = rock_df.sort_index()[rock_df.columns[17:]][0:1]
rock_words = rock_words.sort_values(by=[0], axis=1, ascending=False)
top_word_counts = list(rock_words.loc[0])[0:10]
top_ten = dict(zip(list(rock_words)[0:10], top_word_counts))
top_words['rock'] = top_ten
df_rows.append(list(top_ten.keys()))
df_rows.append(list(top_ten.values()))

romance_df = new_lyric_TF_df[new_lyric_TF_df['category_name']=='romance'].copy()
romance_df.loc[0] = romance_df.sum(numeric_only=True)
romance_words = romance_df.sort_index()[romance_df.columns[17:]][0:1]
romance_words = romance_words.sort_values(by=[0], axis=1, ascending=False)
top_word_counts = list(romance_words.loc[0])[0:10]
top_ten = dict(zip(list(romance_words)[0:10], top_word_counts))
top_words['romance'] = top_ten
df_rows.append(list(top_ten.keys()))
df_rows.append(list(top_ten.values()))

soul_df = new_lyric_TF_df[new_lyric_TF_df['category_name']=='soul'].copy()
soul_df.loc[0] = soul_df.sum(numeric_only=True)
soul_words = soul_df.sort_index()[soul_df.columns[17:]][0:1]
soul_words = soul_words.sort_values(by=[0], axis=1, ascending=False)
top_word_counts = list(soul_words.loc[0])[0:10]
top_ten = dict(zip(list(soul_words)[0:10], top_word_counts))
top_words['soul'] = top_ten
df_rows.append(list(top_ten.keys()))
df_rows.append(list(top_ten.values()))

t1 = time.time()
print(f'Run time: {t1-t0} seconds')

Run time: 0.20601701736450195 seconds


In [40]:
# Print the top words for each category
top_words

{'blues': {'dont know': 122.0,
  'im going': 99.0,
  'hey hey': 93.0,
  'im gonna': 88.0,
  'dont want': 66.0,
  'ooh ooh': 62.0,
  'ive got': 60.0,
  'nah nah': 58.0,
  'aint got': 56.0,
  'baby dont': 54.0},
 'classical': {'oh oh': 26.0,
  'da da': 26.0,
  'ah ah': 24.0,
  'du du': 15.0,
  'dont wanna': 13.0,
  'dont know': 12.0,
  'la la': 10.0,
  'gotta get': 10.0,
  'come come': 8.0,
  'yeah yeah': 7.0},
 'country': {'oh oh': 610.0,
  'dont know': 371.0,
  'na na': 352.0,
  'dont wanna': 270.0,
  'ooh ooh': 267.0,
  'im gonna': 230.0,
  'yeah yeah': 188.0,
  'ive got': 137.0,
  'la la': 137.0,
  'good time': 120.0},
 'funk': {'get get': 251.0,
  'dont stop': 200.0,
  'hey hey': 199.0,
  'oh oh': 195.0,
  'yeah yeah': 116.0,
  'ooh ooh': 86.0,
  'na na': 77.0,
  'oh yeah': 77.0,
  'im gonna': 69.0,
  'la la': 68.0},
 'hiphop': {'yeah yeah': 764.0,
  'na na': 320.0,
  'dont know': 260.0,
  'know im': 214.0,
  'la la': 206.0,
  'uh uh': 201.0,
  'oh oh': 182.0,
  'uh huh': 176.0,
  '

In [41]:
# Create dictionary of top word frequencies
unique_top_words = []
for category in top_words:
    unique_top_words.extend(top_words[category])
unique_top_words = list(set(unique_top_words))

top_word_freq = dict.fromkeys(unique_top_words, 0)
for category in top_words:
    for word in top_words[category]:
        top_word_freq[word]+=1
print(len(top_word_freq))
top_word_freq

51


{'doo doo': 2,
 'nah nah': 1,
 'dont stop': 1,
 'good time': 1,
 'na na': 7,
 'uh uh': 1,
 'uh huh': 1,
 'dont want': 1,
 'hey hey': 4,
 'gotta get': 1,
 'feel like': 1,
 'come back': 1,
 'cant stop': 1,
 'du du': 2,
 'ive got': 4,
 'baby dont': 1,
 'let go': 6,
 'dont wanna': 6,
 'im coming': 1,
 'aint got': 1,
 'youll never': 1,
 'im gonna': 5,
 'come come': 1,
 'wont let': 1,
 'oh yeah': 4,
 'wanna see': 1,
 'love love': 3,
 'ya ya': 1,
 'la la': 9,
 'little bit': 1,
 'go go': 1,
 'baby baby': 1,
 'round round': 1,
 'ooh ooh': 9,
 'cause im': 1,
 'close eyes': 1,
 'oh oh': 13,
 'im going': 1,
 'oh baby': 1,
 'cant get': 1,
 'time time': 2,
 'get get': 2,
 'ooh yeah': 1,
 'ah ah': 3,
 'dont let': 1,
 'da da': 4,
 'yeah yeah': 11,
 'cant see': 1,
 'thats way': 1,
 'know im': 1,
 'dont know': 12}

In [42]:
# Print top words that are not unique to a genre
not_unique = dict(filter(lambda key: key[1] > 1, top_word_freq.items()))
print(len(not_unique))
not_unique

19


{'doo doo': 2,
 'na na': 7,
 'hey hey': 4,
 'du du': 2,
 'ive got': 4,
 'let go': 6,
 'dont wanna': 6,
 'im gonna': 5,
 'oh yeah': 4,
 'love love': 3,
 'la la': 9,
 'ooh ooh': 9,
 'oh oh': 13,
 'time time': 2,
 'get get': 2,
 'ah ah': 3,
 'da da': 4,
 'yeah yeah': 11,
 'dont know': 12}

In [43]:
# Print top words that are unique to one genre
unique_to_one = dict(filter(lambda key: key[1] == 1, top_word_freq.items()))
print(len(unique_to_one))
unique_to_one

32


{'nah nah': 1,
 'dont stop': 1,
 'good time': 1,
 'uh uh': 1,
 'uh huh': 1,
 'dont want': 1,
 'gotta get': 1,
 'feel like': 1,
 'come back': 1,
 'cant stop': 1,
 'baby dont': 1,
 'im coming': 1,
 'aint got': 1,
 'youll never': 1,
 'come come': 1,
 'wont let': 1,
 'wanna see': 1,
 'ya ya': 1,
 'little bit': 1,
 'go go': 1,
 'baby baby': 1,
 'round round': 1,
 'cause im': 1,
 'close eyes': 1,
 'im going': 1,
 'oh baby': 1,
 'cant get': 1,
 'ooh yeah': 1,
 'dont let': 1,
 'cant see': 1,
 'thats way': 1,
 'know im': 1}

In [44]:
# Print top words that are unique to two genre
unique_to_two = dict(filter(lambda key: key[1] == 2, top_word_freq.items()))
print(len(unique_to_two))
unique_to_two

4


{'doo doo': 2, 'du du': 2, 'time time': 2, 'get get': 2}

In [45]:
# Create DataFrame of top ten words
categories = np.array(['blues', 'blues', 'classical', 'classical', 'country', 'country', 'funk', 
              'funk', 'hiphop', 'hiphop', 'indie_alt', 'indie_alt', 'jazz', 'jazz', 'metal',
              'metal', 'pop', 'pop', 'punk', 'punk', 'rnb', 'rnb', 'rock', 'rock', 
              'romance', 'romance', 'soul', 'soul'])
arrays = [categories, np.array(['words', 'frequencies', 'words', 'frequencies', 'words', 
                       'frequencies', 'words', 'frequencies', 'words', 'frequencies', 
                       'words', 'frequencies', 'words', 'frequencies', 'words',
                       'frequencies', 'words', 'frequencies', 'words', 'frequencies', 
                       'words', 'frequencies', 'words', 'frequencies', 'words', 
                       'frequencies', 'words', 'frequencies'])]
tuples = list(zip(*arrays))
index = pd.MultiIndex.from_tuples(tuples, names=['genre', 'w/f'])
top_cooccurring_df = pd.DataFrame(df_rows, index=arrays, columns=range(1,11))
top_cooccurring_df

1          2            3           4   \
blues     words        dont know   im going      hey hey    im gonna   
          frequencies        122         99           93          88   
classical words            oh oh      da da        ah ah       du du   
          frequencies         26         26           24          15   
country   words            oh oh  dont know        na na  dont wanna   
          frequencies        610        371          352         270   
funk      words          get get  dont stop      hey hey       oh oh   
          frequencies        251        200          199         195   
hiphop    words        yeah yeah      na na    dont know     know im   
          frequencies        764        320          260         214   
indie_alt words            oh oh    ooh ooh        la la     doo doo   
          frequencies        361        283          214         169   
jazz      words            na na  thats way  round round      let go   
          frequencies         66         61           53          39   
metal     words            oh oh    hey hey    dont know       da da   
          frequencies        130        119          119         103   
pop       words        time time  come back        oh oh    ooh yeah   
          frequencies          1          1            0           0   
punk      words            oh oh      la la    yeah yeah   dont know   
          frequencies        321        164          158         148   
rnb       words            oh oh  yeah yeah    love love     ooh ooh   
          frequencies        654        580          282         249   
rock      words            oh oh      na na    yeah yeah     doo doo   
          frequencies        876        687          432         428   
romance   words            la la  dont know    yeah yeah   time time   
          frequencies        118        102           85          73   
soul      words            la la      oh oh    yeah yeah     ooh ooh   
          frequencies        358        299          275         235   

                               5           6          7           8   \
blues     words         dont want     ooh ooh    ive got     nah nah   
          frequencies          66          62         60          58   
classical words        dont wanna   dont know      la la   gotta get   
          frequencies          13          12         10          10   
country   words           ooh ooh    im gonna  yeah yeah     ive got   
          frequencies         267         230        188         137   
funk      words         yeah yeah     ooh ooh      na na     oh yeah   
          frequencies         116          86         77          77   
hiphop    words             la la       uh uh      oh oh      uh huh   
          frequencies         206         201        182         176   
indie_alt words         dont know   yeah yeah      ah ah  dont wanna   
          frequencies         150         131        103          94   
jazz      words          dont let       oh oh  dont know     get get   
          frequencies          39          35         33          33   
metal     words            let go     ive got      go go    cant see   
          frequencies          99          92         91          91   
pop       words         im coming   wanna see  cant stop    wont let   
          frequencies           0           0          0           0   
punk      words             na na     ive got    ooh ooh    im gonna   
          frequencies         140         112        106         104   
rnb       words         dont know  dont wanna    oh yeah      let go   
          frequencies         205         178        171         137   
rock      words         dont know       la la      da da     ooh ooh   
          frequencies         339         334        282         266   
romance   words         love love     ooh ooh      oh oh    im gonna   
          frequencies          68          67         64          55 

In [14]:
# Save DataFrame to CSV
top_cooccurring_df.to_csv('../../Data/Analysis/top_cooccurring_words.csv')

# Conclusions about top co-occurring words
- From the total 140 top ten co-occurring words from each of the 14 genres, 51 were unique
- 19 of the top ten co-occurring words were not unique to a genre
- 32 of the top ten co-occurring words are unique to one genre
- 4 of the top ten co-occurring words are unique to two genre
    - 'get get' is unique to the funk and jazz genres
    - 'time time' is unique to the pop and romance genres
    - 'doo doo' is unique to the indie_alt and rock genres
    - 'du du' is unique to the classical and soul genres
- 'oh oh' is in the top ten co-occurring words for every genre except blues and pop
- 'dont know' is in the top ten co-occurring words for every genre except funk and pop
- 'yeah yeah' is in the top ten co-occurring words for 11 out of 14 genres